In [13]:
import pandas as pd
import glob
import statsmodels.api as sm
from matplotlib import pyplot as plt
import datetime as dt
#import datatable
from multiprocessing import Process
#import pyarrow


Read all the CSVs into a single dataframe, in a single line. Cool!

but this didn't convert to datetime

df = pd.concat(map(pd.read_csv, glob.glob('//scdata2/signalshar/Data_Analysis/INRIX_API/Speed_Data/Daily_Download/*.csv')))

In [84]:
#read all csv files into dataframe and parse dates in 26 seconds. Awesome. 
folder = r'//scdata2/signalshar/Data_Analysis/INRIX_API/Speed_Data/Daily_Download/'
all_files = glob.glob(folder + "*.parquet")
li = []

for filename in all_files:
    df = pd.read_parquet(filename) #, parse_dates=['Date Time'], header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
del li
frame

,Date Time,Segment ID,Speed(miles/hour),Ref Speed(miles/hour),Travel Time(Minutes),CValue
0,2021-11-01 00:00:00,120645869,15,15,0.27,NaN
1,2021-11-01 00:15:00,120645869,15,15,0.27,NaN
2,2021-11-01 00:30:00,120645869,15,15,0.27,NaN
3,2021-11-01 00:45:00,120645869,15,15,0.27,NaN
4,2021-11-01 01:00:00,120645869,15,15,0.27,NaN
...,...,...,...,...,...,...
14012995,2021-12-06 22:45:00,1237021323,40,31,0.07,98.67
14012996,2021-12-06 23:00:00,1237021323,37,31,0.07,86.67
14012997,2021-12-06 23:15:00,1237021323,33,31,0.08,100.00
14012998,2021-12-06 23:30:00,1237021323,39,31,0.07,98.67


In [96]:
# Extract reference speed for each segment, and create list of uniqe segments for loop
speed = frame.rename(columns={'Segment ID' : 'XDSegID'}).drop(columns = ['CValue', 'Speed(miles/hour)', 'Travel Time(Minutes)'])
segments = set(speed.XDSegID)
speed = speed.groupby('XDSegID').mean()
print(len(segments))
print(len(speed))

4050
4050


In [104]:
##THIS IS A CIRCULAR BS MESS, CAN ONLY BE RUN ONCE

dim_signals_XD=pd.read_csv('//scdata2/signalshar/Data_Analysis/Data/Performance/dim_signals_XD.csv', index_col='XDSegID')
print(len(dim_signals_XD.index))
dim_signals_XD = dim_signals_XD.merge(speed, on='XDSegID', how='inner')
dim_signals_XD['reference_travel_time'] = dim_signals_XD['Miles'].div(dim_signals_XD['Ref Speed(miles/hour)']).mul(60)
print(len(dim_signals_XD.index))
dim_signals_XD.to_csv('//scdata2/signalshar/Data_Analysis/Data/Performance/dim_signals_XD.csv')#, index=False)
dim_signals_XD


4314
4314


,OID,PreviousXD,NextXDSegI,FRC,ToExport,Miles,Lanes,RoadNumber,RoadName,Country,...,Latitude,Longitude,key,_merge,AADT,SPEED,travel_time_minutes,group,Ref Speed(miles/hour),reference_travel_time
XDSegID,,,,,,,,,,,,,,,,,,,,,
116337789,6408164,NaN,NaN,5,NaN,0.111931,1.296,NaN,NE GOODFELLOW ST,UNITED STATES,...,44.024750,-116.943451,14037-116337789,left_only,NaN,NaN,NaN,1,11,0.610534
120916743,6520609,NaN,NaN,5,NaN,0.080331,1.068,NaN,SE GOODFELLOW ST,UNITED STATES,...,44.024750,-116.943451,14037-120916743,left_only,NaN,NaN,NaN,2,9,0.535540
383985787,8663597,1.187523e+09,4.289573e+08,2,NaN,0.422287,3.164,30.0,E IDAHO AVE,UNITED STATES,...,44.024750,-116.943451,14037-383985787,left_only,23600.000000,35.0,0.723921,3,28,0.904901
383985787,8663597,1.187523e+09,4.289573e+08,2,NaN,0.422287,3.164,30.0,E IDAHO AVE,UNITED STATES,...,44.024735,-116.947056,14036-383985787,left_only,23600.000000,35.0,0.723921,3,28,0.904901
383985787,8663597,1.187523e+09,4.289573e+08,2,NaN,0.422287,3.164,30.0,E IDAHO AVE,UNITED STATES,...,44.024745,-116.939955,14003-383985787,left_only,23600.000000,35.0,0.723921,3,28,0.904901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237054703,11217290,1.236841e+09,1.236877e+09,2,NaN,0.654716,2.847,101.0,OREGON COAST HWY,UNITED STATES,...,42.050533,-124.260003,07049-1237054703,left_only,16766.666667,45.0,0.872955,2706,43,0.913557
1237059921,11219924,4.409257e+08,3.860463e+08,2,NaN,0.461936,3.530,39.0,S 6TH ST,UNITED STATES,...,42.206496,-121.732391,11018-1237059921,left_only,25866.666667,35.0,0.791891,1204,29,0.955730
1237059921,11219924,4.409257e+08,3.860463e+08,2,NaN,0.461936,3.530,39.0,S 6TH ST,UNITED STATES,...,42.206515,-121.730093,11013-1237059921,left_only,25866.666667,35.0,0.791891,1204,29,0.955730


In [99]:
data = pd.DataFrame([])
#segments=[1237022307, 440963878]
for s in segments:
    df = frame[frame['Segment ID'] == s].set_index('Date Time').resample('H').mean().ffill()
    comp = sm.tsa.seasonal_decompose(x=df['Travel Time(Minutes)'], period=24*7, two_sided=False) #use .values to override pandas frequency, bc T is not recognized in sm
    df = df.join(comp.trend).join(comp.seasonal).join(comp.resid)
    df['resid zscore'] = (df['resid'] - df['resid'].mean()).div(df['resid'].std())
    df['trend pct_change'] = df.trend.pct_change()
    df['trend pct_change zscore'] = (df['trend pct_change'] - df['trend pct_change'].mean()).div(df['trend pct_change'].std())
    #df['trend pct_change cusum'] = df['trend pct_change'].cumsum()
    #df.dropna(inplace=True)
    #comp.plot()
    #plt.show()
    #df['trend pct_change zscore'].plot()
    #plt.show()
    #print(df.head())
    #print(comp.seasonal)
    data = data.append(df.dropna())

data

,Segment ID,Speed(miles/hour),Ref Speed(miles/hour),Travel Time(Minutes),CValue,trend,seasonal,resid,resid zscore,trend pct_change,trend pct_change zscore
Date Time,,,,,,,,,,,
2021-11-08 01:00:00,1.237051e+09,28.00,28.0,1.1300,84.000,1.163140,-0.029682,-0.003458,-0.022271,-2.220446e-16,0.026912
2021-11-08 02:00:00,1.237051e+09,28.00,28.0,1.1300,84.000,1.163140,-0.029682,-0.003458,-0.022271,0.000000e+00,0.026912
2021-11-08 03:00:00,1.237051e+09,28.00,28.0,1.1275,99.000,1.163132,0.011318,-0.046951,-0.669310,-6.396889e-06,0.013379
2021-11-08 04:00:00,1.237051e+09,30.00,28.0,1.0600,90.000,1.163065,-0.070182,-0.032884,-0.460036,-5.757237e-05,-0.094887
2021-11-08 05:00:00,1.237051e+09,32.75,28.0,0.9650,96.670,1.162909,-0.183668,-0.014241,-0.182686,-1.343433e-04,-0.257303
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-06 19:00:00,3.859743e+08,13.00,13.0,0.2725,100.000,0.256168,0.010389,0.005942,0.353705,8.714344e-05,0.164766
2021-12-06 20:00:00,3.859743e+08,11.00,13.0,0.3250,91.665,0.256198,0.057388,0.011414,0.693368,1.161811e-04,0.218724
2021-12-06 21:00:00,3.859743e+08,13.75,13.0,0.2550,90.000,0.256153,0.002883,-0.004037,-0.265763,-1.742514e-04,-0.320958


In [102]:
data.to_parquet('//scdata2/signalshar/Data_Analysis/Data/Performance/travel_time.parquet')